In [68]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import make_pipeline
from scipy import stats
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTEENN
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, make_scorer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from mlens.ensemble import SuperLearner
import pickle

In [69]:
train_data_values = pd.read_csv('./data/train_values.csv', index_col=0)
train_data_labels = pd.read_csv('./data/train_labels.csv', index_col=0)

train_data = pd.concat([train_data_values, train_data_labels], axis=1)

In [70]:
train_data.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,damage_grade
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,2
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,3
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,2
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,3


In [71]:
train_data.columns

Index(['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
       'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'land_surface_condition', 'foundation_type', 'roof_type',
       'ground_floor_type', 'other_floor_type', 'position',
       'plan_configuration', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'legal_ownership_status', 'count_families', 'has_secondary_use',
       'has_secondary_use_agriculture', 'has_secondary_use_hotel',
       'has_secondary_use_rental', 'has_secondary_use_institution',
       'has_secondary_use_school', 'has_secondary_use_industry',
     

In [72]:
numeric_features = [
  'count_floors_pre_eq', 'age', 
  'area_percentage', 'height_percentage',
  'count_families'
]

categorical_features = [
  'land_surface_condition', 'foundation_type',
  'roof_type', 'ground_floor_type',
  'other_floor_type', 'position',
  'plan_configuration', 'legal_ownership_status'
]

other_features = list(np.setdiff1d(
  train_data.columns,
  categorical_features + ['damage_grade'],
  assume_unique=True
))

Let's see how imbalanced the data really is:

In [73]:
train_data[train_data['damage_grade'] == 1].shape

(25124, 39)

In [74]:
train_data[train_data['damage_grade'] == 2].shape

(148259, 39)

In [75]:
train_data[train_data['damage_grade'] == 3].shape

(87218, 39)

Let's see if we can improve the imbalance by removing outliers. 

In [76]:
train_data = train_data[(np.abs(stats.zscore(train_data[numeric_features])) < 3).all(axis=1)]

In [77]:
train_data[train_data['damage_grade'] == 1].shape

(22900, 39)

In [78]:
train_data[train_data['damage_grade'] == 2].shape

(142343, 39)

In [79]:
train_data[train_data['damage_grade'] == 3].shape

(84588, 39)

It has helped a little bit, but it also helps indirectly with balancing the data by other techniques later.

In [80]:
X_train, X_test, y_train, y_test = train_test_split(train_data_values,
                                                    train_data_labels['damage_grade'],
                                                    test_size=0.25,
                                                    random_state=999)

In [81]:
X_train.head(10)

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
985502,6,673,2937,2,5,13,6,t,i,x,...,0,0,0,0,0,0,0,0,0,0
140510,3,1387,2791,5,100,15,10,t,r,q,...,0,1,0,0,0,0,0,0,0,0
701507,11,765,3900,2,20,11,7,t,r,n,...,0,0,0,0,0,0,0,0,0,0
438410,17,1313,2490,3,0,8,6,t,r,q,...,0,0,0,0,0,0,0,0,0,0
800610,21,1219,2948,2,0,6,5,t,r,q,...,0,0,0,0,0,0,0,0,0,0
152270,22,763,4002,1,10,12,3,t,r,n,...,0,0,0,0,0,0,0,0,0,0
511298,16,1399,4589,1,30,6,3,n,r,n,...,0,0,0,0,0,0,0,0,0,0
42065,10,337,296,2,5,5,5,t,r,q,...,0,0,0,0,0,0,0,0,0,0
669566,13,1154,11118,2,20,7,4,n,r,n,...,0,0,0,0,0,0,0,0,0,0


In [82]:
preprocessor = ColumnTransformer(
  transformers=[
      ('ohe', OneHotEncoder(drop="first"), categorical_features)
  ],
  remainder='passthrough'
)

In [83]:
X_train = pd.DataFrame(
  preprocessor.fit_transform(X_train),
  index=X_train.index,
  columns=(
    list(preprocessor.named_transformers_['ohe'].get_feature_names(categorical_features)) +
    other_features
  )
)

X_test = pd.DataFrame(preprocessor.transform(X_test),
                      index=X_test.index,
                      columns=X_train.columns)

In [84]:
X_train.head(10)

,land_surface_condition_o,land_surface_condition_t,foundation_type_i,foundation_type_r,foundation_type_u,foundation_type_w,roof_type_q,roof_type_x,ground_floor_type_m,ground_floor_type_v,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
985502,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
140510,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
701507,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
438410,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
800610,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
152270,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
511298,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
42065,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
669566,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
sm = SMOTE(random_state=27)

In [86]:
X_train, y_train = sm.fit_sample(X_train, y_train)

In [20]:
parameters = { 
  'min_samples_split': [5, 10],
  'n_estimators': [100, 150]
}

In [21]:
rfc = GridSearchCV(
  RandomForestClassifier(), 
  parameters, 
  cv=5,
  scoring=make_scorer(f1_score, average='micro'),
  verbose=2
).fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] min_samples_split=5, n_estimators=100 ...........................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............ min_samples_split=5, n_estimators=100, total= 1.1min
[CV] min_samples_split=5, n_estimators=100 ...........................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.1min remaining:    0.0s


[CV] ............ min_samples_split=5, n_estimators=100, total= 1.1min
[CV] min_samples_split=5, n_estimators=100 ...........................
[CV] ............ min_samples_split=5, n_estimators=100, total= 1.0min
[CV] min_samples_split=5, n_estimators=100 ...........................
[CV] ............ min_samples_split=5, n_estimators=100, total= 1.0min
[CV] min_samples_split=5, n_estimators=100 ...........................
[CV] ............ min_samples_split=5, n_estimators=100, total= 1.1min
[CV] min_samples_split=5, n_estimators=150 ...........................
[CV] ............ min_samples_split=5, n_estimators=150, total= 1.7min
[CV] min_samples_split=5, n_estimators=150 ...........................
[CV] ............ min_samples_split=5, n_estimators=150, total= 1.6min
[CV] min_samples_split=5, n_estimators=150 ...........................
[CV] ............ min_samples_split=5, n_estimators=150, total= 1.6min
[CV] min_samples_split=5, n_estimators=150 ...........................
[CV] .

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 26.5min finished


In [22]:
dummy = DummyClassifier(strategy='uniform').fit(X_train, y_train)

In [23]:
rfc_pred = rfc.predict(X_test)

In [24]:
dummy_pred = dummy.predict(X_test)

In [25]:
f1_score(y_test, rfc_pred, average='micro')

0.7154456570121718

In [26]:
f1_score(y_test, dummy_pred, average='micro')

0.3321975103989194

In [27]:
rfc.best_params_

{'min_samples_split': 5, 'n_estimators': 150}

In [28]:
rfc.best_estimator_.feature_importances_

array([4.12697544e-03, 1.34810288e-02, 1.48282469e-02, 5.58917837e-02,
       7.12132740e-03, 1.57045446e-02, 1.81129986e-02, 2.44310584e-02,
       4.94440893e-04, 3.40336518e-02, 9.13055031e-03, 7.27331681e-04,
       2.86526569e-02, 5.89886443e-03, 1.01790457e-02, 9.01005288e-04,
       1.68189435e-02, 1.62162891e-02, 2.45230148e-04, 4.56663325e-03,
       1.14111400e-05, 3.00304655e-05, 5.99546937e-05, 1.35121410e-04,
       2.91398000e-03, 2.84988168e-04, 2.28845934e-03, 9.48629233e-04,
       4.89757230e-03, 1.38871781e-03, 1.19122582e-01, 8.80213876e-02,
       8.42721671e-02, 3.61010965e-02, 7.27864144e-02, 6.36890294e-02,
       4.75152927e-02, 9.86192223e-03, 5.67590125e-02, 7.08064322e-03,
       3.58430938e-03, 8.63231908e-03, 2.14022479e-02, 1.86385007e-02,
       9.54495662e-03, 8.64149719e-03, 5.23642474e-03, 3.25112912e-03,
       2.05913881e-02, 8.80682209e-03, 5.30510040e-03, 4.05387948e-03,
       1.06703636e-03, 1.66313150e-04, 6.80921112e-05, 2.04605176e-04,
      

In [29]:
X_test.columns

Index(['land_surface_condition_o', 'land_surface_condition_t',
       'foundation_type_i', 'foundation_type_r', 'foundation_type_u',
       'foundation_type_w', 'roof_type_q', 'roof_type_x',
       'ground_floor_type_m', 'ground_floor_type_v', 'ground_floor_type_x',
       'ground_floor_type_z', 'other_floor_type_q', 'other_floor_type_s',
       'other_floor_type_x', 'position_o', 'position_s', 'position_t',
       'plan_configuration_c', 'plan_configuration_d', 'plan_configuration_f',
       'plan_configuration_m', 'plan_configuration_n', 'plan_configuration_o',
       'plan_configuration_q', 'plan_configuration_s', 'plan_configuration_u',
       'legal_ownership_status_r', 'legal_ownership_status_v',
       'legal_ownership_status_w', 'geo_level_1_id', 'geo_level_2_id',
       'geo_level_3_id', 'count_floors_pre_eq', 'age', 'area_percentage',
       'height_percentage', 'has_superstructure_adobe_mud',
       'has_superstructure_mud_mortar_stone', 'has_superstructure_stone_flag',
    

In [30]:
feature_importantance_series = pd.Series(rfc.best_estimator_.feature_importances_)

In [31]:
feature_importantance_series.index = X_test.columns

In [32]:
feature_importantance_series.sort_values(ascending=False)

geo_level_1_id                            0.119123
geo_level_2_id                            0.088021
geo_level_3_id                            0.084272
age                                       0.072786
area_percentage                           0.063689
has_superstructure_mud_mortar_stone       0.056759
foundation_type_r                         0.055892
height_percentage                         0.047515
count_floors_pre_eq                       0.036101
ground_floor_type_v                       0.034034
other_floor_type_q                        0.028653
roof_type_x                               0.024431
has_superstructure_cement_mortar_brick    0.021402
count_families                            0.020591
has_superstructure_timber                 0.018639
roof_type_q                               0.018113
position_s                                0.016819
position_t                                0.016216
foundation_type_w                         0.015705
foundation_type_i              

In [95]:
xgb_parameters = {
  'max_depth': [6, 10],
  'colsample_bytree': [0.8]
}

In [96]:
xgb_model = GridSearchCV(
  XGBClassifier(), 
  xgb_parameters, 
  cv=5,
  scoring=make_scorer(f1_score, average='micro'),
  verbose=2
).fit(X_train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] colsample_bytree=0.8, max_depth=6 ...............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................ colsample_bytree=0.8, max_depth=6, total= 6.1min
[CV] colsample_bytree=0.8, max_depth=6 ...............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  6.1min remaining:    0.0s


[CV] ................ colsample_bytree=0.8, max_depth=6, total= 6.2min
[CV] colsample_bytree=0.8, max_depth=6 ...............................
[CV] ................ colsample_bytree=0.8, max_depth=6, total= 6.4min
[CV] colsample_bytree=0.8, max_depth=6 ...............................
[CV] ................ colsample_bytree=0.8, max_depth=6, total= 6.2min
[CV] colsample_bytree=0.8, max_depth=6 ...............................
[CV] ................ colsample_bytree=0.8, max_depth=6, total= 6.2min
[CV] colsample_bytree=0.8, max_depth=10 ..............................
[CV] ............... colsample_bytree=0.8, max_depth=10, total= 9.3min
[CV] colsample_bytree=0.8, max_depth=10 ..............................
[CV] ............... colsample_bytree=0.8, max_depth=10, total= 9.6min
[CV] colsample_bytree=0.8, max_depth=10 ..............................
[CV] ............... colsample_bytree=0.8, max_depth=10, total= 9.3min
[CV] colsample_bytree=0.8, max_depth=10 ..............................
[CV] .

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 77.5min finished


In [97]:
xgb_model.best_params_

{'colsample_bytree': 0.8, 'max_depth': 10}

In [98]:
xgb_model.best_estimator_.feature_importances_

array([0.0051761 , 0.00898156, 0.0269817 , 0.35768974, 0.01765117,
       0.01960185, 0.01617406, 0.01229804, 0.00468889, 0.04411929,
       0.00520313, 0.00319394, 0.04038239, 0.00439255, 0.00741334,
       0.00229994, 0.02620419, 0.01077505, 0.00310347, 0.00574895,
       0.        , 0.00519479, 0.00327846, 0.00330452, 0.00405099,
       0.00249583, 0.00676117, 0.00264473, 0.00439563, 0.0039854 ,
       0.04047594, 0.00817027, 0.00366049, 0.02061346, 0.01205816,
       0.00760393, 0.0078385 , 0.01316659, 0.06037288, 0.02069468,
       0.00621894, 0.01279929, 0.02414555, 0.01093778, 0.00836611,
       0.00564653, 0.01229437, 0.00542065, 0.01404187, 0.01610699,
       0.00522421, 0.0042778 , 0.00442137, 0.00244279, 0.00073108,
       0.00164371, 0.00313185, 0.        , 0.00530979, 0.00399351],
      dtype=float32)

In [99]:
feature_importantance_series = pd.Series(xgb_model.best_estimator_.feature_importances_)
feature_importantance_series.index = X_test.columns
feature_importantance_series.sort_values(ascending=False)

foundation_type_r                         0.357690
has_superstructure_mud_mortar_stone       0.060373
ground_floor_type_v                       0.044119
geo_level_1_id                            0.040476
other_floor_type_q                        0.040382
foundation_type_i                         0.026982
position_s                                0.026204
has_superstructure_cement_mortar_brick    0.024146
has_superstructure_stone_flag             0.020695
count_floors_pre_eq                       0.020613
foundation_type_w                         0.019602
foundation_type_u                         0.017651
roof_type_q                               0.016174
has_secondary_use                         0.016107
count_families                            0.014042
has_superstructure_adobe_mud              0.013167
has_superstructure_mud_mortar_brick       0.012799
roof_type_x                               0.012298
has_superstructure_rc_engineered          0.012294
age                            

In [100]:
xgb_pred = xgb_model.predict(X_test.as_matrix())

/Users/jackyho/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [101]:
f1_score(y_test, xgb_pred, average='micro')

0.722153151908643

In [89]:
light_parameters = {
  'num_leaves': [50, 80, 150],
  'min_data_in_leaf': [50, 100, 300],
  'max_depth': [5, 10]
}

In [90]:
light_model = GridSearchCV(
  LGBMClassifier(), 
  light_parameters, 
  cv=5,
  scoring=make_scorer(f1_score, average='micro'),
  verbose=2
).fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV] max_depth=5, min_data_in_leaf=50, num_leaves=50 .................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .. max_depth=5, min_data_in_leaf=50, num_leaves=50, total=  17.4s
[CV] max_depth=5, min_data_in_leaf=50, num_leaves=50 .................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   17.4s remaining:    0.0s


[CV] .. max_depth=5, min_data_in_leaf=50, num_leaves=50, total=  17.4s
[CV] max_depth=5, min_data_in_leaf=50, num_leaves=50 .................
[CV] .. max_depth=5, min_data_in_leaf=50, num_leaves=50, total=  17.8s
[CV] max_depth=5, min_data_in_leaf=50, num_leaves=50 .................
[CV] .. max_depth=5, min_data_in_leaf=50, num_leaves=50, total=  16.5s
[CV] max_depth=5, min_data_in_leaf=50, num_leaves=50 .................
[CV] .. max_depth=5, min_data_in_leaf=50, num_leaves=50, total=  19.1s
[CV] max_depth=5, min_data_in_leaf=50, num_leaves=80 .................
[CV] .. max_depth=5, min_data_in_leaf=50, num_leaves=80, total=  18.6s
[CV] max_depth=5, min_data_in_leaf=50, num_leaves=80 .................
[CV] .. max_depth=5, min_data_in_leaf=50, num_leaves=80, total=  18.8s
[CV] max_depth=5, min_data_in_leaf=50, num_leaves=80 .................
[CV] .. max_depth=5, min_data_in_leaf=50, num_leaves=80, total=  17.8s
[CV] max_depth=5, min_data_in_leaf=50, num_leaves=80 .................
[CV] .

[CV]  max_depth=10, min_data_in_leaf=50, num_leaves=150, total=  39.6s
[CV] max_depth=10, min_data_in_leaf=100, num_leaves=50 ...............
[CV]  max_depth=10, min_data_in_leaf=100, num_leaves=50, total=  27.6s
[CV] max_depth=10, min_data_in_leaf=100, num_leaves=50 ...............
[CV]  max_depth=10, min_data_in_leaf=100, num_leaves=50, total=  26.0s
[CV] max_depth=10, min_data_in_leaf=100, num_leaves=50 ...............
[CV]  max_depth=10, min_data_in_leaf=100, num_leaves=50, total=  27.0s
[CV] max_depth=10, min_data_in_leaf=100, num_leaves=50 ...............
[CV]  max_depth=10, min_data_in_leaf=100, num_leaves=50, total=  25.5s
[CV] max_depth=10, min_data_in_leaf=100, num_leaves=50 ...............
[CV]  max_depth=10, min_data_in_leaf=100, num_leaves=50, total=  26.4s
[CV] max_depth=10, min_data_in_leaf=100, num_leaves=80 ...............
[CV]  max_depth=10, min_data_in_leaf=100, num_leaves=80, total=  28.9s
[CV] max_depth=10, min_data_in_leaf=100, num_leaves=80 ...............
[CV]  

[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed: 34.6min finished


In [91]:
feature_importantance_series = pd.Series(light_model.best_estimator_.feature_importances_)
feature_importantance_series.index = X_test.columns
feature_importantance_series.sort_values(ascending=False)

geo_level_2_id                            7895
geo_level_1_id                            6902
geo_level_3_id                            5436
area_percentage                           3016
age                                       2881
height_percentage                         1955
count_floors_pre_eq                       1250
count_families                            1105
has_superstructure_mud_mortar_stone        977
has_superstructure_timber                  928
roof_type_q                                923
position_s                                 636
land_surface_condition_t                   634
has_superstructure_cement_mortar_brick     623
other_floor_type_q                         586
foundation_type_r                          541
ground_floor_type_v                        519
has_secondary_use                          517
ground_floor_type_x                        504
other_floor_type_x                         481
has_superstructure_bamboo                  442
has_superstru

In [92]:
light_model.best_params_

{'max_depth': 10, 'min_data_in_leaf': 50, 'num_leaves': 150}

In [93]:
light_pred = light_model.predict(X_test)

In [94]:
f1_score(y_test, light_pred, average='micro')

0.7130358705161854

In [26]:
ada_parameters = {
  'n_estimators': [50, 100, 200],
  'learning_rate': [0.1, 1, 2]
}

In [27]:
ada_model = GridSearchCV(
  AdaBoostClassifier(), 
  ada_parameters, 
  cv=5,
  scoring=make_scorer(f1_score, average='micro'),
  verbose=2
).fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] learning_rate=0.1, n_estimators=50 ..............................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............... learning_rate=0.1, n_estimators=50, total=  33.7s
[CV] learning_rate=0.1, n_estimators=50 ..............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   33.7s remaining:    0.0s


[CV] ............... learning_rate=0.1, n_estimators=50, total=  33.8s
[CV] learning_rate=0.1, n_estimators=50 ..............................
[CV] ............... learning_rate=0.1, n_estimators=50, total=  34.4s
[CV] learning_rate=0.1, n_estimators=50 ..............................
[CV] ............... learning_rate=0.1, n_estimators=50, total=  33.5s
[CV] learning_rate=0.1, n_estimators=50 ..............................
[CV] ............... learning_rate=0.1, n_estimators=50, total=  32.8s
[CV] learning_rate=0.1, n_estimators=100 .............................
[CV] .............. learning_rate=0.1, n_estimators=100, total= 1.2min
[CV] learning_rate=0.1, n_estimators=100 .............................
[CV] .............. learning_rate=0.1, n_estimators=100, total= 1.1min
[CV] learning_rate=0.1, n_estimators=100 .............................
[CV] .............. learning_rate=0.1, n_estimators=100, total= 1.1min
[CV] learning_rate=0.1, n_estimators=100 .............................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed: 55.5min finished


In [29]:
feature_importantance_series = pd.Series(ada_model.best_estimator_.feature_importances_)
feature_importantance_series.index = X_test.columns
feature_importantance_series.sort_values(ascending=False)

geo_level_1_id                            0.245
age                                       0.075
area_percentage                           0.075
roof_type_q                               0.070
count_floors_pre_eq                       0.070
count_families                            0.060
other_floor_type_q                        0.050
land_surface_condition_t                  0.050
position_t                                0.030
position_s                                0.030
foundation_type_r                         0.025
has_secondary_use                         0.025
height_percentage                         0.020
geo_level_2_id                            0.020
ground_floor_type_v                       0.015
other_floor_type_x                        0.015
has_superstructure_mud_mortar_stone       0.015
has_superstructure_timber                 0.015
has_superstructure_bamboo                 0.015
has_superstructure_mud_mortar_brick       0.010
has_superstructure_cement_mortar_brick  

In [30]:
ada_model.best_params_

{'learning_rate': 1, 'n_estimators': 200}

In [31]:
ada_pred = ada_model.predict(X_test)
f1_score(y_test, ada_pred, average='micro')

0.6477414007459594

Ada did the worst among the first layer models.

## Gather all the models for stacking

In [102]:
rf_model = RandomForestClassifier(min_samples_split = 5, n_estimators = 150).fit(X_train, y_train)
xgb_model = XGBClassifier(colsample_bytree = 0.8, max_depth = 10).fit(X_train, y_train)
light_model = LGBMClassifier(min_data_in_leaf = 50, num_leaves = 150, max_depth = 10).fit(X_train, y_train)

In [103]:
stacking_train_df = pd.DataFrame({
 'rf_pred': rf_model.predict(X_train),
  'xgb_pred': xgb_model.predict(X_train),
  'light_pred': light_model.predict(X_train)
})

In [104]:
stacking_test_df = pd.DataFrame({
 'rf_pred': rf_model.predict(X_test),
  'xgb_pred': xgb_model.predict(X_test.as_matrix()),
  'light_pred': light_model.predict(X_test)
})

/Users/jackyho/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [105]:
stacking_train_df

,rf_pred,xgb_pred,light_pred
0,1,1,1
1,2,2,2
2,3,3,2
3,3,3,3
4,3,3,3
...,...,...,...
333886,3,3,3
333887,3,3,3
333888,3,3,3
333889,3,3,3


In [106]:
cat_parameters = {
  'depth': [6, 7, 8, 9],
  'l2_leaf_reg': [10, 20],
  'one_hot_max_size': [10]
}

In [107]:
cat_model = GridSearchCV(
  CatBoostClassifier(), 
  cat_parameters, 
  cv=5,
  scoring=make_scorer(f1_score, average='micro'),
  verbose=2
).fit(stacking_train_df, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] depth=6, l2_leaf_reg=10, one_hot_max_size=10 ....................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0:	learn: 1.0431843	total: 56.5ms	remaining: 56.5s
1:	learn: 0.9924762	total: 97.4ms	remaining: 48.6s
2:	learn: 0.9458567	total: 140ms	remaining: 46.4s
3:	learn: 0.9028179	total: 185ms	remaining: 46.2s
4:	learn: 0.8629304	total: 226ms	remaining: 45s
5:	learn: 0.8258750	total: 279ms	remaining: 46.3s
6:	learn: 0.7913029	total: 325ms	remaining: 46.1s
7:	learn: 0.7590038	total: 373ms	remaining: 46.2s
8:	learn: 0.7287420	total: 418ms	remaining: 46s
9:	learn: 0.7003208	total: 460ms	remaining: 45.5s
10:	learn: 0.6735879	total: 506ms	remaining: 45.5s
11:	learn: 0.6483951	total: 550ms	remaining: 45.3s
12:	learn: 0.6246282	total: 598ms	remaining: 45.4s
13:	learn: 0.6021539	total: 641ms	remaining: 45.1s
14:	learn: 0.5808919	total: 680ms	remaining: 44.6s
15:	learn: 0.5607461	total: 725ms	remaining: 44.6s
16:	learn: 0.5416342	total: 776ms	remaining: 44.9s
17:	learn: 0.5234869	total: 813ms	remaining: 44.3s
18:	learn: 0.5062387	total: 850ms	remaining: 43.9s
19:	learn: 0.4898269	total: 893ms	remaining

166:	learn: 0.1007565	total: 8.11s	remaining: 40.5s
167:	learn: 0.1006876	total: 8.16s	remaining: 40.4s
168:	learn: 0.1006261	total: 8.2s	remaining: 40.3s
169:	learn: 0.1005668	total: 8.24s	remaining: 40.2s
170:	learn: 0.1005046	total: 8.29s	remaining: 40.2s
171:	learn: 0.1004438	total: 8.34s	remaining: 40.1s
172:	learn: 0.1003856	total: 8.38s	remaining: 40.1s
173:	learn: 0.1003313	total: 8.42s	remaining: 40s
174:	learn: 0.1002774	total: 8.47s	remaining: 39.9s
175:	learn: 0.1002261	total: 8.51s	remaining: 39.9s
176:	learn: 0.1001752	total: 8.57s	remaining: 39.8s
177:	learn: 0.1001260	total: 8.62s	remaining: 39.8s
178:	learn: 0.1000791	total: 8.66s	remaining: 39.7s
179:	learn: 0.1000329	total: 8.71s	remaining: 39.7s
180:	learn: 0.0999888	total: 8.76s	remaining: 39.6s
181:	learn: 0.0999501	total: 8.81s	remaining: 39.6s
182:	learn: 0.0999089	total: 8.86s	remaining: 39.5s
183:	learn: 0.0998692	total: 8.91s	remaining: 39.5s
184:	learn: 0.0998343	total: 8.95s	remaining: 39.4s
185:	learn: 0.0

327:	learn: 0.0987216	total: 15.9s	remaining: 32.6s
328:	learn: 0.0987206	total: 16s	remaining: 32.6s
329:	learn: 0.0987202	total: 16s	remaining: 32.6s
330:	learn: 0.0987192	total: 16.1s	remaining: 32.5s
331:	learn: 0.0987182	total: 16.2s	remaining: 32.5s
332:	learn: 0.0987181	total: 16.2s	remaining: 32.5s
333:	learn: 0.0987181	total: 16.3s	remaining: 32.4s
334:	learn: 0.0987172	total: 16.3s	remaining: 32.4s
335:	learn: 0.0987169	total: 16.4s	remaining: 32.4s
336:	learn: 0.0987164	total: 16.4s	remaining: 32.3s
337:	learn: 0.0987156	total: 16.5s	remaining: 32.3s
338:	learn: 0.0987148	total: 16.6s	remaining: 32.3s
339:	learn: 0.0987143	total: 16.6s	remaining: 32.3s
340:	learn: 0.0987135	total: 16.7s	remaining: 32.2s
341:	learn: 0.0987127	total: 16.7s	remaining: 32.2s
342:	learn: 0.0987121	total: 16.8s	remaining: 32.2s
343:	learn: 0.0987113	total: 16.9s	remaining: 32.2s
344:	learn: 0.0987105	total: 16.9s	remaining: 32.1s
345:	learn: 0.0987098	total: 17s	remaining: 32.1s
346:	learn: 0.0987

...

323:	learn: 0.1381826	total: 21.9s	remaining: 45.7s
324:	learn: 0.1381823	total: 22s	remaining: 45.7s
325:	learn: 0.1381820	total: 22.1s	remaining: 45.7s
326:	learn: 0.1381817	total: 22.2s	remaining: 45.7s
327:	learn: 0.1381813	total: 22.3s	remaining: 45.7s
328:	learn: 0.1381808	total: 22.4s	remaining: 45.6s
329:	learn: 0.1381807	total: 22.5s	remaining: 45.6s
330:	learn: 0.1381804	total: 22.6s	remaining: 45.6s
331:	learn: 0.1381801	total: 22.6s	remaining: 45.5s
332:	learn: 0.1381799	total: 22.7s	remaining: 45.5s
333:	learn: 0.1381796	total: 22.8s	remaining: 45.5s
334:	learn: 0.1381792	total: 22.9s	remaining: 45.4s
335:	learn: 0.1381791	total: 23s	remaining: 45.4s
336:	learn: 0.1381787	total: 23s	remaining: 45.3s
337:	learn: 0.1381784	total: 23.1s	remaining: 45.3s
338:	learn: 0.1381781	total: 23.2s	remaining: 45.3s
339:	learn: 0.1381781	total: 23.3s	remaining: 45.2s
340:	learn: 0.1381781	total: 23.3s	remaining: 45.1s
341:	learn: 0.1381778	total: 23.4s	remaining: 45.1s
342:	learn: 0.1381

483:	learn: 0.1381532	total: 35.2s	remaining: 37.5s
484:	learn: 0.1381531	total: 35.3s	remaining: 37.5s
485:	learn: 0.1381529	total: 35.4s	remaining: 37.4s
486:	learn: 0.1381529	total: 35.4s	remaining: 37.3s
487:	learn: 0.1381528	total: 35.5s	remaining: 37.3s
488:	learn: 0.1381527	total: 35.6s	remaining: 37.2s
489:	learn: 0.1381525	total: 35.8s	remaining: 37.2s
490:	learn: 0.1381524	total: 35.8s	remaining: 37.2s
491:	learn: 0.1381523	total: 35.9s	remaining: 37.1s
492:	learn: 0.1381522	total: 36s	remaining: 37s
493:	learn: 0.1381521	total: 36.1s	remaining: 37s
494:	learn: 0.1381520	total: 36.2s	remaining: 37s
495:	learn: 0.1381519	total: 36.3s	remaining: 36.9s
496:	learn: 0.1381518	total: 36.4s	remaining: 36.9s
497:	learn: 0.1381517	total: 36.5s	remaining: 36.8s
498:	learn: 0.1381516	total: 36.6s	remaining: 36.7s
499:	learn: 0.1381515	total: 36.7s	remaining: 36.7s
500:	learn: 0.1381514	total: 36.8s	remaining: 36.6s
501:	learn: 0.1381513	total: 36.8s	remaining: 36.6s
502:	learn: 0.138151

644:	learn: 0.1381417	total: 49.8s	remaining: 27.4s
645:	learn: 0.1381417	total: 49.9s	remaining: 27.4s
646:	learn: 0.1381416	total: 50s	remaining: 27.3s
647:	learn: 0.1381416	total: 50.1s	remaining: 27.2s
648:	learn: 0.1381415	total: 50.2s	remaining: 27.2s
649:	learn: 0.1381415	total: 50.3s	remaining: 27.1s
650:	learn: 0.1381414	total: 50.4s	remaining: 27s
651:	learn: 0.1381414	total: 50.5s	remaining: 27s
652:	learn: 0.1381414	total: 50.6s	remaining: 26.9s
653:	learn: 0.1381413	total: 50.8s	remaining: 26.9s
654:	learn: 0.1381413	total: 50.9s	remaining: 26.8s
655:	learn: 0.1381412	total: 51s	remaining: 26.7s
656:	learn: 0.1381412	total: 51.1s	remaining: 26.7s
657:	learn: 0.1381411	total: 51.2s	remaining: 26.6s
658:	learn: 0.1381411	total: 51.3s	remaining: 26.5s
659:	learn: 0.1381411	total: 51.3s	remaining: 26.4s
660:	learn: 0.1381410	total: 51.4s	remaining: 26.4s
661:	learn: 0.1381410	total: 51.5s	remaining: 26.3s
662:	learn: 0.1381409	total: 51.6s	remaining: 26.2s
663:	learn: 0.138140

804:	learn: 0.1381364	total: 1m 3s	remaining: 15.3s
805:	learn: 0.1381364	total: 1m 3s	remaining: 15.2s
806:	learn: 0.1381364	total: 1m 3s	remaining: 15.1s
807:	learn: 0.1381364	total: 1m 3s	remaining: 15s
808:	learn: 0.1381363	total: 1m 3s	remaining: 15s
809:	learn: 0.1381363	total: 1m 3s	remaining: 14.9s
810:	learn: 0.1381363	total: 1m 3s	remaining: 14.8s
811:	learn: 0.1381363	total: 1m 3s	remaining: 14.7s
812:	learn: 0.1381362	total: 1m 3s	remaining: 14.7s
813:	learn: 0.1381362	total: 1m 3s	remaining: 14.6s
814:	learn: 0.1381362	total: 1m 3s	remaining: 14.5s
815:	learn: 0.1381362	total: 1m 3s	remaining: 14.4s
816:	learn: 0.1381362	total: 1m 4s	remaining: 14.3s
817:	learn: 0.1381361	total: 1m 4s	remaining: 14.3s
818:	learn: 0.1381361	total: 1m 4s	remaining: 14.2s
819:	learn: 0.1381361	total: 1m 4s	remaining: 14.1s
820:	learn: 0.1381361	total: 1m 4s	remaining: 14s
821:	learn: 0.1381360	total: 1m 4s	remaining: 13.9s
822:	learn: 0.1381360	total: 1m 4s	remaining: 13.9s
823:	learn: 0.1381

961:	learn: 0.1381335	total: 1m 15s	remaining: 3s
962:	learn: 0.1381334	total: 1m 15s	remaining: 2.92s
963:	learn: 0.1381334	total: 1m 16s	remaining: 2.84s
964:	learn: 0.1381334	total: 1m 16s	remaining: 2.76s
965:	learn: 0.1381334	total: 1m 16s	remaining: 2.68s
966:	learn: 0.1381334	total: 1m 16s	remaining: 2.6s
967:	learn: 0.1381334	total: 1m 16s	remaining: 2.53s
968:	learn: 0.1381334	total: 1m 16s	remaining: 2.46s
969:	learn: 0.1381333	total: 1m 16s	remaining: 2.38s
970:	learn: 0.1381333	total: 1m 17s	remaining: 2.3s
971:	learn: 0.1381333	total: 1m 17s	remaining: 2.22s
972:	learn: 0.1381333	total: 1m 17s	remaining: 2.14s
973:	learn: 0.1381333	total: 1m 17s	remaining: 2.07s
974:	learn: 0.1381333	total: 1m 17s	remaining: 1.99s
975:	learn: 0.1381333	total: 1m 17s	remaining: 1.92s
976:	learn: 0.1381332	total: 1m 18s	remaining: 1.84s
977:	learn: 0.1381332	total: 1m 18s	remaining: 1.76s
978:	learn: 0.1381332	total: 1m 18s	remaining: 1.68s
979:	learn: 0.1381332	total: 1m 18s	remaining: 1.6s

In [108]:
cat_model.best_params_

{'depth': 6, 'l2_leaf_reg': 10, 'one_hot_max_size': 10}

In [109]:
cat_pred = cat_model.predict(stacking_test_df)
f1_score(y_test, cat_pred, average='micro')

0.7166275268223051

## Let's put all these together

In [123]:
from mlens.metrics import make_scorer
accuracy_scorer = make_scorer(f1_score, average='micro')

In [124]:
ensemble = SuperLearner(scorer=accuracy_scorer, random_state=123, verbose=2, folds=5)

In [125]:
ensemble.add([
  RandomForestClassifier(min_samples_split=5, n_estimators=150, random_state=123), 
  XGBClassifier(colsample_bytree=0.8, max_depth=10, seed=123),
  LGBMClassifier(min_data_in_leaf=50, num_leaves=150, max_depth=10, random_state=123)
])

SuperLearner(array_check=None, backend=None, folds=5,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=3582, shuffle=False,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=5, raise_on_ex...e=micro))],
   n_jobs=-1, name='group-7', raise_on_exception=True, transformers=[])],
   verbose=1)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=123, sample_size=20,
       scorer=make_scorer(f1_score, average=micro), shuffle=False,
       verbose=2)

In [126]:
ensemble.add_meta(CatBoostClassifier(
  random_seed=123,
  depth=6, 
  l2_leaf_reg=10, 
  one_hot_max_size=10
))

SuperLearner(array_check=None, backend=None, folds=5,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=3582, shuffle=False,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=5, raise_on_ex...e=micro))],
   n_jobs=-1, name='group-8', raise_on_exception=True, transformers=[])],
   verbose=1)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=123, sample_size=20,
       scorer=make_scorer(f1_score, average=micro), shuffle=False,
       verbose=2)

In [127]:
sub_preprocessor = ColumnTransformer(
  transformers=[('ohe', OneHotEncoder(drop="first"), categorical_features)],
  remainder='passthrough'
)

In [128]:
X = sub_preprocessor.fit_transform(train_data_values)

In [129]:
sm = SMOTE(random_state=27)

X_train, y_train = sm.fit_sample(X, train_data_labels['damage_grade'])

In [130]:
ensemble.fit(X_train, y_train)


Fitting 2 layers
Processing layer-1             

/Users/jackyho/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [lgbmclassifier.0.5] Could not score lgbmclassifier. Details:
TypeError("__call__() missing 1 required positional argument: 'y_true'")
  (name, inst_name, exc), MetricWarning)
/Users/jackyho/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [lgbmclassifier.0.4] Could not score lgbmclassifier. Details:
TypeError("__call__() missing 1 required positional argument: 'y_true'")
  (name, inst_name, exc), MetricWarning)
/Users/jackyho/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [lgbmclassifier.0.1] Could not score lgbmclassifier. Details:
TypeError("__call__() missing 1 required positional argument: 'y_true'")
  (name, inst_name, exc), MetricWarning)
/Users/jackyho/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [lgbmclassifier.0.3] Could not score lgbmcla

done | 00:30:25
Processing layer-2             0:	learn: 1.0740649	total: 112ms	remaining: 1m 52s
1:	learn: 1.0513140	total: 186ms	remaining: 1m 32s
2:	learn: 1.0299701	total: 275ms	remaining: 1m 31s
3:	learn: 1.0102860	total: 356ms	remaining: 1m 28s
4:	learn: 0.9918310	total: 455ms	remaining: 1m 30s
5:	learn: 0.9746641	total: 543ms	remaining: 1m 29s
6:	learn: 0.9585075	total: 638ms	remaining: 1m 30s
7:	learn: 0.9432495	total: 727ms	remaining: 1m 30s
8:	learn: 0.9289214	total: 819ms	remaining: 1m 30s
9:	learn: 0.9156205	total: 904ms	remaining: 1m 29s
10:	learn: 0.9030108	total: 986ms	remaining: 1m 28s
11:	learn: 0.8910507	total: 1.07s	remaining: 1m 28s
12:	learn: 0.8798622	total: 1.16s	remaining: 1m 28s
13:	learn: 0.8693014	total: 1.25s	remaining: 1m 27s
14:	learn: 0.8592311	total: 1.33s	remaining: 1m 27s
15:	learn: 0.8497095	total: 1.43s	remaining: 1m 27s
16:	learn: 0.8407863	total: 1.51s	remaining: 1m 27s
17:	learn: 0.8322523	total: 1.6s	remaining: 1m 27s
18:	learn: 0.8242404	total: 

159:	learn: 0.6654334	total: 13s	remaining: 1m 8s
160:	learn: 0.6654166	total: 13.1s	remaining: 1m 8s
161:	learn: 0.6654127	total: 13.2s	remaining: 1m 8s
162:	learn: 0.6653965	total: 13.3s	remaining: 1m 8s
163:	learn: 0.6653812	total: 13.3s	remaining: 1m 8s
164:	learn: 0.6653748	total: 13.4s	remaining: 1m 7s
165:	learn: 0.6653605	total: 13.5s	remaining: 1m 7s
166:	learn: 0.6653572	total: 13.6s	remaining: 1m 7s
167:	learn: 0.6653542	total: 13.6s	remaining: 1m 7s
168:	learn: 0.6653373	total: 13.7s	remaining: 1m 7s
169:	learn: 0.6653212	total: 13.8s	remaining: 1m 7s
170:	learn: 0.6653058	total: 13.9s	remaining: 1m 7s
171:	learn: 0.6652910	total: 14s	remaining: 1m 7s
172:	learn: 0.6652796	total: 14.1s	remaining: 1m 7s
173:	learn: 0.6652699	total: 14.1s	remaining: 1m 7s
174:	learn: 0.6652567	total: 14.2s	remaining: 1m 7s
175:	learn: 0.6652441	total: 14.3s	remaining: 1m 7s
176:	learn: 0.6652354	total: 14.4s	remaining: 1m 6s
177:	learn: 0.6652317	total: 14.5s	remaining: 1m 6s
178:	learn: 0.66

319:	learn: 0.6649685	total: 25.1s	remaining: 53.3s
320:	learn: 0.6649681	total: 25.2s	remaining: 53.3s
321:	learn: 0.6649677	total: 25.3s	remaining: 53.2s
322:	learn: 0.6649676	total: 25.3s	remaining: 53.1s
323:	learn: 0.6649675	total: 25.4s	remaining: 53s
324:	learn: 0.6649669	total: 25.5s	remaining: 52.9s
325:	learn: 0.6649669	total: 25.5s	remaining: 52.8s
326:	learn: 0.6649661	total: 25.6s	remaining: 52.7s
327:	learn: 0.6649661	total: 25.7s	remaining: 52.6s
328:	learn: 0.6649661	total: 25.7s	remaining: 52.5s
329:	learn: 0.6649661	total: 25.8s	remaining: 52.4s
330:	learn: 0.6649660	total: 25.9s	remaining: 52.3s
331:	learn: 0.6649657	total: 26s	remaining: 52.3s
332:	learn: 0.6649649	total: 26.1s	remaining: 52.2s
333:	learn: 0.6649649	total: 26.2s	remaining: 52.1s
334:	learn: 0.6649649	total: 26.2s	remaining: 52.1s
335:	learn: 0.6649648	total: 26.3s	remaining: 52s
336:	learn: 0.6649648	total: 26.4s	remaining: 51.9s
337:	learn: 0.6649641	total: 26.5s	remaining: 51.9s
338:	learn: 0.6649

...

799:	learn: 0.6649345	total: 1m 5s	remaining: 16.4s
800:	learn: 0.6649345	total: 1m 5s	remaining: 16.3s
801:	learn: 0.6649345	total: 1m 5s	remaining: 16.2s
802:	learn: 0.6649345	total: 1m 5s	remaining: 16.1s
803:	learn: 0.6649345	total: 1m 5s	remaining: 16s
804:	learn: 0.6649345	total: 1m 5s	remaining: 16s
805:	learn: 0.6649344	total: 1m 5s	remaining: 15.9s
806:	learn: 0.6649344	total: 1m 6s	remaining: 15.8s
807:	learn: 0.6649344	total: 1m 6s	remaining: 15.7s
808:	learn: 0.6649344	total: 1m 6s	remaining: 15.6s
809:	learn: 0.6649344	total: 1m 6s	remaining: 15.6s
810:	learn: 0.6649344	total: 1m 6s	remaining: 15.5s
811:	learn: 0.6649344	total: 1m 6s	remaining: 15.4s
812:	learn: 0.6649344	total: 1m 6s	remaining: 15.3s
813:	learn: 0.6649344	total: 1m 6s	remaining: 15.2s
814:	learn: 0.6649344	total: 1m 6s	remaining: 15.2s
815:	learn: 0.6649344	total: 1m 6s	remaining: 15.1s
816:	learn: 0.6649344	total: 1m 6s	remaining: 15s
817:	learn: 0.6649344	total: 1m 7s	remaining: 14.9s
818:	learn: 0.6649

958:	learn: 0.6649340	total: 1m 19s	remaining: 3.38s
959:	learn: 0.6649340	total: 1m 19s	remaining: 3.3s
960:	learn: 0.6649340	total: 1m 19s	remaining: 3.22s
961:	learn: 0.6649340	total: 1m 19s	remaining: 3.14s
962:	learn: 0.6649340	total: 1m 19s	remaining: 3.05s
963:	learn: 0.6649340	total: 1m 19s	remaining: 2.97s
964:	learn: 0.6649340	total: 1m 19s	remaining: 2.89s
965:	learn: 0.6649340	total: 1m 19s	remaining: 2.81s
966:	learn: 0.6649340	total: 1m 19s	remaining: 2.72s
967:	learn: 0.6649340	total: 1m 19s	remaining: 2.64s
968:	learn: 0.6649340	total: 1m 20s	remaining: 2.56s
969:	learn: 0.6649340	total: 1m 20s	remaining: 2.48s
970:	learn: 0.6649340	total: 1m 20s	remaining: 2.4s
971:	learn: 0.6649340	total: 1m 20s	remaining: 2.31s
972:	learn: 0.6649340	total: 1m 20s	remaining: 2.23s
973:	learn: 0.6649340	total: 1m 20s	remaining: 2.15s
974:	learn: 0.6649340	total: 1m 20s	remaining: 2.06s
975:	learn: 0.6649340	total: 1m 20s	remaining: 1.98s
976:	learn: 0.6649340	total: 1m 20s	remaining: 1

/Users/jackyho/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:226: MetricWarning: [catboostclassifier.0.0] Could not score catboostclassifier. Details:
TypeError("__call__() missing 1 required positional argument: 'y_true'")
  (name, inst_name, exc), MetricWarning)


done | 00:01:24
Fit complete                        | 00:31:51


SuperLearner(array_check=None, backend=None, folds=5,
       layers=[Layer(backend='threading', dtype=<class 'numpy.float32'>, n_jobs=-1,
   name='layer-1', propagate_features=None, raise_on_exception=True,
   random_state=3582, shuffle=False,
   stack=[Group(backend='threading', dtype=<class 'numpy.float32'>,
   indexer=FoldIndex(X=None, folds=5, raise_on_ex...e=micro))],
   n_jobs=-1, name='group-8', raise_on_exception=True, transformers=[])],
   verbose=1)],
       model_selection=False, n_jobs=None, raise_on_exception=True,
       random_state=123, sample_size=20,
       scorer=make_scorer(f1_score, average=micro), shuffle=False,
       verbose=2)

In [131]:
test_data_values = pd.read_csv('./data/test_values.csv', index_col=0)

In [132]:
X_test = sub_preprocessor.transform(test_data_values)

In [133]:
preds = ensemble.predict(X_test)


Predicting 2 layers


/Users/jackyho/anaconda3/lib/python3.7/site-packages/mlens/parallel/_base_functions.py:313: ParameterChangeWarning: Parameter value ('multi:softprob') has changed since model was fitted ('binary:logistic').
  (lpar, rpar), ParameterChangeWarning)


Processing layer-1             done | 00:00:11
Processing layer-2             done | 00:00:00
Predict complete                    | 00:00:12


In [134]:
sub = pd.DataFrame({
  'building_id': test_data_values.index,
  'damage_grade': map(int, preds)
})

In [135]:
sub.to_csv('final-submission', index=False)

In [137]:
pickle.dump(ensemble, open('ensemble_model.pickle', 'wb'))